<a href="https://colab.research.google.com/github/johntzwei/metric-statistical-advantage/blob/main/bvnd_wmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: to run a sizable number of bootstrap simulations requires quite a bit of compute. We have set up our code with multiprocessing, so we recommend using colab to connect to a local instance with lots of cpu cores.

# Imports

In [ ]:
!pip install sacrebleu
!pip install gdown

In [ ]:
import pickle
import itertools
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from multiprocessing import Pool
import sacrebleu
import pdb

# Data

In [ ]:
!gdown --id 1RZg1GbRYvyXHCv4R3lzoEiVCgPC2e9TT

Downloading...
From: https://drive.google.com/uc?id=1RZg1GbRYvyXHCv4R3lzoEiVCgPC2e9TT
To: /home/johnny/wmt16-19_toen_scored.pkl
301MB [00:04, 61.2MB/s]


In [ ]:
wmt_scored = pickle.load(open('wmt16-19_toen_scored.pkl', 'rb'))
wmt_scored.head(1)

,lp,HITId,WorkerId,score,time,system,type,sid,reference,source,...,chrf:statistics_10,chrf:statistics_11,chrf:statistics_12,chrf:statistics_13,chrf:statistics_14,chrf:statistics_15,chrf:statistics_16,chrf:statistics_17,bert_score:f1,bleurt_score
0,de-en,3QQUBC64ZEEFIPT2SKINDZZTZ3SNXN,A0077,78,2052.0,online-G,SYSTEM,1906,Rather than having an executive make the annou...,Anstatt einen Manager die Ankündigung machen z...,...,151,95,145,150,85,144,149,76,0.941347,-0.00404


# Bias variance noise decomposition

In [ ]:
# adopted from https://github.com/rasbt/mlxtend/blob/master/mlxtend/evaluate/bias_variance_decomp.py
def bias_var_noise_decomposition(metric_results, human_results, no_bias=False):
    main_predictions = np.apply_along_axis(lambda x:
                                           np.argmax(np.bincount(x)),
                                           axis=0,
                                           arr=metric_results)

    optimal_predictions = np.apply_along_axis(lambda x:
                                       np.argmax(np.bincount(x)),
                                       axis=0,
                                       arr=human_results)
    
    if no_bias:
        main_predictions = optimal_predictions
    
    avg_expected_loss = (human_results != metric_results).mean()

    noises = (human_results != optimal_predictions).mean(axis=0)
    probs = (metric_results == optimal_predictions).mean(axis=0)

    avg_noise_contrib = ((2 * probs - 1) * noises).mean()

    avg_bias_contrib = (main_predictions != optimal_predictions).mean()

    signs = (main_predictions == optimal_predictions).astype(np.int64) * 2 - 1
    variances = (metric_results != main_predictions).mean(axis=0)
    avg_var_contrib = (signs * variances).mean()

    return avg_expected_loss, avg_bias_contrib, avg_var_contrib, avg_noise_contrib

In [ ]:
def pairs(x):
    for (year, lp), group in x.groupby(['year', 'lp']):
        systems = group.system.unique()
        
        for i, j in itertools.combinations(systems, 2):
            yield (year, lp), i, j
            
all_pairs = list(pairs(wmt_scored))
pairs_2019 = list(pairs(wmt_scored[wmt_scored.year == 2019]))

# all the pairwise predictions in the WMT dataset
# in the form of (year, language pair), system1, system2
all_pairs[:10]

[((2016, 'cs-en'), 'online-B', 'uedin-nmt'),
 ((2016, 'cs-en'), 'online-B', 'PJATK'),
 ((2016, 'cs-en'), 'online-B', 'online-A'),
 ((2016, 'cs-en'), 'online-B', 'jhu-pbmt'),
 ((2016, 'cs-en'), 'online-B', 'cu-mergedtrees'),
 ((2016, 'cs-en'), 'uedin-nmt', 'PJATK'),
 ((2016, 'cs-en'), 'uedin-nmt', 'online-A'),
 ((2016, 'cs-en'), 'uedin-nmt', 'jhu-pbmt'),
 ((2016, 'cs-en'), 'uedin-nmt', 'cu-mergedtrees'),
 ((2016, 'cs-en'), 'PJATK', 'online-A')]

In [ ]:
# get pairwise predictions from the scoring of each system
# in the form of a (num_pairs,) length vector, where 1 or 0 denotes the ordering of that pair
def get_preds(pairs, scores):
    preds = np.zeros(len(pairs))
    if type(scores) == type({}):
        for ii, ((year, lp), i, j) in enumerate(pairs):
            preds[ii] = 1 if scores[year, lp, i] - scores[year, lp, j] > 0 else 0
    else:
        for ii, ((year, lp), i, j) in enumerate(pairs):
            preds[ii] = 1 if scores.loc[year, lp, i] - scores.loc[year, lp, j] > 0 else 0
            
    return preds

In [ ]:
groupby_cached = [ (i, pd.DataFrame(g)) for i, g in wmt_scored.groupby(['year', 'lp', 'system']) ]
groupby_labels = [ i[0] for i in groupby_cached ]

### Experimental parameters

In [ ]:
NUM_WORKERS = 12
NUM_TRIALS = 10000
CHUNKSIZE = 10
METRICS = ['true_preds', 'human', 'bleurt', 'bert_score', 'bleu', 'ter', 'chrf']

In [ ]:
SEED = 0
random_state = np.random.RandomState(SEED)

### Optimal predictions (human main predictions)

In [ ]:
def f(seed):
    this_rs = np.random.RandomState(seed)

    # using bootstrap, sample frac=1 number of human judgments for each MT system
    # in WMT we typically have 1k-2k judgments per system collected
    groups = [ g.sample(frac=1, replace=True, random_state=this_rs) for i, g in groupby_cached ]

    # calculate the observed means for each MT system from this bootstrap sampling
    observed_means = [ g['score'].mean() for g in groups ]
    d = { label:mean for label, mean in zip(groupby_labels, observed_means) }

    # generate the pairwise predictions from this bootstrap sampling
    return get_preds(all_pairs, d)

with Pool(NUM_WORKERS) as p:
    it = tqdm(p.imap_unordered(f, [ random_state.randint(0, 2**32 - 1) for i in range(0, NUM_TRIALS) ],
                               chunksize=CHUNKSIZE), total=NUM_TRIALS)
    output = list(it)
    human_results = np.array(output, dtype=np.int64)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
# optimal predictions
optimal_predictions = np.apply_along_axis(lambda x:
                                   np.argmax(np.bincount(x)),
                                   axis=0,
                                   arr=human_results)

### Bootstrap experiments

In [ ]:
# for metrics such as BLEU, TER, chrF, the system level statistics are not averages
# of their segment level scores - we need to calculate aggregate segment statistics 
# (precomputed in the pickle) to compute the eventual system level scores

# thanks to sacrebleu for easy to use open source implementation of these metrics

def bleu_agg(x):
    cols = [ ('bleu:counts_%d' % i) for i in range(0, 4) ] + [ ('bleu:totals_%d' % i) for i in range(0, 4) ]
    cols += ['bleu:sys_len', 'bleu:ref_len']
    aggs = x[cols].sum()

    totals = [ aggs['bleu:totals_%d' % i] for i in range(0, 4) ]
    counts = [ aggs['bleu:counts_%d' % i] for i in range(0, 4) ]
    ref_len = aggs['bleu:ref_len']
    sys_len = aggs['bleu:sys_len']

    return sacrebleu.compute_bleu(counts, totals, sys_len, ref_len).score

def ter_agg(x):
    cols = [ 'ter:num_edits', 'ter:ref_length' ]
    aggs = x[cols].sum()

    num_edits = aggs['ter:num_edits']
    ref_length = aggs['ter:ref_length']

    # for one reference
    return -num_edits / ref_length

def chrf_agg(x):
    cols = [ ('chrf:statistics_%d' % i) for i in range(0, 18) ]
    aggs = x[cols].sum()

    statistics = [ aggs['chrf:statistics_%d' % i] for i in range(0, 18) ]

    return sacrebleu.CHRF.compute_chrf(statistics, order=6, beta=2).score

def bert_score_agg(x):
    return x['bert_score:f1'].mean()

def bleurt_agg(x):
    return x['bleurt_score'].mean()

def human_agg(x):
    return x['score'].mean()

# dummy function
def true_preds_agg(x):
  return None

In [ ]:
def f(x):
    metric, seed = x
    this_rs = np.random.RandomState(seed)

    # compute human scores
    groups = [ g.sample(frac=1, replace=True, random_state=this_rs) for i, g in groupby_cached ]
    observed_means = [ g['score'].mean() for g in groups ]
    observed_means = { label:mean for label, mean in zip(groupby_labels, observed_means) }
    
    # resampled so humans and metrics are independent
    # this is here to keep consistency when comparing with the human estimator
    groups = [ g.sample(frac=1, replace=True, random_state=this_rs) for i, g in groupby_cached ]

    # get agg function
    agg = globals()['%s_agg' % metric]

    # compute metric scores
    agg_means = [ agg(g) for g in groups ]
    agg_means = { label:mean for label, mean in zip(groupby_labels, agg_means) }

    if metric == 'bleurt':
      human_preds = get_preds(pairs_2019, observed_means)
      metric_preds = get_preds(pairs_2019, agg_means)
    elif metric == 'true_preds':
      human_preds = get_preds(all_pairs, observed_means)
      metric_preds = optimal_predictions  # constant, computed from optimal predictions above
    else:
      human_preds = get_preds(all_pairs, observed_means)
      metric_preds = get_preds(all_pairs, agg_means)

    return human_preds, metric_preds

bootstrap_results = {}
with Pool(NUM_WORKERS) as p:
    for metric in tqdm(METRICS):
        it = tqdm(p.imap_unordered(f, [ (metric, random_state.randint(0, 2**32 - 1)) for i in range(0, NUM_TRIALS) ],
                                   chunksize=CHUNKSIZE), total=NUM_TRIALS)
        output = list(it)

        human_results = np.array([ i[0] for i in output ], dtype=np.int64)
        metric_results = np.array([ i[1] for i in output ], dtype=np.int64)
        bootstrap_results[metric] = (human_results, metric_results)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

### Decomposition

In [ ]:
decomp_results = {}
for metric in METRICS:
  no_bias = metric in ['human', 'true_preds']
  metric_results, human_results = bootstrap_results[metric]
  decomp_results[metric] = bias_var_noise_decomposition(metric_results, human_results, no_bias=no_bias)

# Results

### Jsons

In [ ]:
json.dumps(decomp_results)

'{"true_preds": [0.04662092145015106, 0.0, 0.04662092145015106, 0.0], "human": [0.06517129909365559, 0.0, 0.04655611782477341, 0.018614978731117823], "bleurt": [0.1283695550351288, 0.10772833723653395, 0.016108665105386416, 0.004564333302107728], "bert_score": [0.10236465256797583, 0.08610271903323263, 0.013156495468277945, 0.003103892386706949], "bleu": [0.14132953172205437, 0.1268882175226586, 0.006915634441087613, 0.007546726676737158], "ter": [0.1840821752265861, 0.1729607250755287, 0.00864237160120846, 0.0025213024773413904], "chrf": [0.12387673716012085, 0.10498489425981873, 0.009377794561933534, 0.009491543821752265]}'

### Table

In [ ]:
print(
r'''
\begin{table*}[!t]
    \small
    \begin{tabular}{r c | c c c }
    \toprule
    & & \multicolumn{3}{c}{Error components} \\
    & $\text{Err}_{\text{obs}}(\cdot)$ & $c_0\text{Noise}$ & $\text{Bias}$ & $c_1\text{Var}$  \\
    \midrule
'''
)

names = {
    'true_preds' : "Optimal ($\Delta^{H*}_{S,S'}$)",
    'human' : "Human ($\widehat{\Delta^{H}_{S,S'}}$)",
    'bert_score' : "\sc{BERTscore}",
    'chrf' : "\sc{chrF}",
    'bleurt' : "\sc{BLEURT}**",
    'bleu' : "\sc{BLEU}",
    'ter' : "\sc{TER}"
}

sorted_results = sorted(list(decomp_results.items()), key=lambda x: x[1][0])
for metric, (avg_expected_loss, avg_bias_contrib, avg_var_contrib, avg_noise_contrib) in sorted_results:
    max_idx = np.argmax([avg_noise_contrib, avg_bias_contrib, avg_var_contrib])
    if max_idx == 0:
        s = '%s & %.3f & \\bf{%.3f} & %.3f & %.3f \\\\'
    elif max_idx == 1:
        s = '%s & %.3f & %.3f & \\bf{%.3f}& %.3f \\\\'
    else:
        s = '%s & %.3f & %.3f & %.3f & \\bf{%.3f} \\\\'
    print(s % (names[metric], avg_expected_loss, (avg_expected_loss-avg_bias_contrib-avg_var_contrib), avg_bias_contrib, avg_var_contrib))

print(
r'''
    \end{tabular}%
    \end{minipage}
    \caption{Decomposition of the pairwise error of different metrics (left: WMT, right: SummEval). Highlighted in bold is the largest error component. 10K boostrap trials are conducted for estimation of the expectations (estimation error $<10^{-3}$). *Denotes an estimator assumed to be unbiased in the simulation. **{\sc BLEURT} is evaluated only on WMT2019. ***{\sc SUPERT} is a reference-less metric.}
    \label{table:bvd_wmt}
\end{table*}
'''
)


\begin{table*}[!t]
    \small
    \begin{tabular}{r c | c c c }
    \toprule
    & & \multicolumn{3}{c}{Error components} \\
    & $\text{Err}_{\text{obs}}(\cdot)$ & $c_0\text{Noise}$ & $\text{Bias}$ & $c_1\text{Var}$  \\
    \midrule

Optimal ($\Delta^{H*}_{S,S'}$) & 0.047 & 0.000 & 0.000 & \bf{0.047} \\
Human ($\widehat{\Delta^{H}_{S,S'}}$) & 0.065 & 0.019 & 0.000 & \bf{0.047} \\
\sc{BERTscore} & 0.102 & 0.003 & \bf{0.086}& 0.013 \\
\sc{chrF} & 0.124 & 0.010 & \bf{0.105}& 0.009 \\
\sc{BLEURT}** & 0.128 & 0.005 & \bf{0.108}& 0.016 \\
\sc{BLEU} & 0.141 & 0.008 & \bf{0.127}& 0.007 \\
\sc{TER} & 0.184 & 0.002 & \bf{0.173}& 0.009 \\

    \end{tabular}%
    \end{minipage}
    \caption{Decomposition of the pairwise error of different metrics (left: WMT, right: SummEval). Highlighted in bold is the largest error component. 10K boostrap trials are conducted for estimation of the expectations (estimation error $<10^{-3}$). *Denotes an estimator assumed to be unbiased in the simulation. **{\sc